# VacationPy

In [1]:
import pandas as pd
import numpy as np
import json
import requests
import time
from pprint import pprint 

import matplotlib.pyplot as plt
import seaborn as sns

# Import API key
from api_keys import weather_api_key

import gmaps
from api_keys import g_key
gmaps.configure(api_key=g_key)

In [3]:
filepath = "cities_new.csv"
df = pd.read_csv(filepath)
#df = df.drop(['Unnamed: 0'], axis=1)
df = df.rename(columns={"City Name": "City"})
df.head()

,City ID,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,0,Punta Arenas,-53.15,-70.92,46.40,81,90,8.05
1,1,Bengkulu,-3.80,102.27,75.29,86,100,4.03
2,2,Severo-Kuril'sk,50.68,156.12,38.79,73,98,31.65
3,3,Lillooet,50.69,-121.94,50.85,93,90,8.05
4,4,Tahoua,14.89,5.27,82.40,23,0,4.70


In [4]:
coords = df[["Latitude", "Longitude"]]
humids = df["Humidity"]

# Customize the size of the figure
figure_layout = {
    'width': '1200px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type = "SATELLITE")

heat = gmaps.heatmap_layer(coords, weights = humids, dissipating=False, max_intensity = 500)
# Add the layer to the map
fig.add_layer(heat)

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [ ]:
mask = (df["Temperature"] >= 70) & (df["Temperature"] <= 78) & (df["Wind Speed"] <= 7) & (df["Cloudiness"] == 0)
ideal_weather_cities = df.loc[mask].reset_index(drop=True)
ideal_weather_cities

In [ ]:
#inits
hotels = []
countries = []

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "rankby": "distance",
    "distance" : 5000,
    "type": "hotel",
    "key": g_key,
}

# use iterrows to iterate through pandas dataframe
for index, row in ideal_weather_cities.iterrows():
    
    city_name = row["City"]
    location = f"{row.Latitude}, {row.Longitude}"

    # add location to params dict
    params["location"] = location

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    #pprint(results)
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
    
        hotel = results[0]['name']
        country = results[0]['plus_code']['compound_code'].split(",")[1]
        
        hotels.append(hotel)
        countries.append(country)
        
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    time.sleep(1)
        
print("------------")

ideal_weather_cities["Hotel Name"] = hotels
ideal_weather_cities["Country"] = countries
ideal_weather_cities

In [ ]:
list_info = []

for indx, row in ideal_weather_cities.iterrows():
    info = f"City: {row.City}<br>-----------<br>Hotel: {row['Hotel Name']}<br>-----------<br>Country: {row.Country}"
    list_info.append(info)

In [ ]:
coords = df[["Latitude", "Longitude"]]
coords_ideal = ideal_weather_cities[["Latitude", "Longitude"]]
humids = df["Humidity"]

# Customize the size of the figure
figure_layout = {
    'width': '1200px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, map_type = "SATELLITE")

# Assign the marker layer to a variable
markers = gmaps.marker_layer(coords_ideal, info_box_content=list_info)
# Add the layer to the map
fig.add_layer(markers)

heat = gmaps.heatmap_layer(coords, weights = humids, dissipating=False, max_intensity=500)
# Add the layer to the map
fig.add_layer(heat)

fig